In [100]:
# Importing all the necessary libraries
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

## Importing and preprocessing the cocktail dataset 

dataset: https://www.kaggle.com/datasets/aadyasingh55/cocktails

In [4]:
df = pd.read_csv('data/final_cocktails.csv', index_col=0)
df.head()

,id,name,alcoholic,category,glassType,instructions,drinkThumbnail,ingredients,ingredientMeasures,text
0,0,A1,Alcoholic,Cocktail,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grand Marnier', 'Lemon Juice', 'Grena...","['1 3/4 shot ', '1 Shot ', '1/4 Shot', '1/8 Sh...",question Generate a cocktail with Gin Grand Ma...
1,1,ABC,Alcoholic,Shot,Shot glass,Layered in a shot glass.,https://www.thecocktaildb.com/images/media/dri...,"['Amaretto', 'Baileys irish cream', 'Cognac']","['1/3 ', '1/3 ', '1/3 ']",question Generate a cocktail with Amaretto Bai...
2,2,Ace,Alcoholic,Cocktail,Martini Glass,Shake all the ingredients in a cocktail shaker...,https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grenadine', 'Heavy cream', 'Milk', 'E...","['2 shots ', '1/2 shot ', '1/2 shot ', '1/2 sh...",question Generate a cocktail with Gin Grenadin...
3,3,ACID,Alcoholic,Shot,Shot glass,Poor in the 151 first followed by the 101 serv...,https://www.thecocktaildb.com/images/media/dri...,"['151 proof rum', 'Wild Turkey']","['1 oz Bacardi ', '1 oz ']",question Generate a cocktail with 151 proof ru...
4,4,Adam,Alcoholic,Ordinary Drink,Cocktail glass,"In a shaker half-filled with ice cubes, combin...",https://www.thecocktaildb.com/images/media/dri...,"['Dark rum', 'Lemon juice', 'Grenadine']","['2 oz ', '1 oz ', '1 tsp ']",question Generate a cocktail with Dark rum Lem...


In [12]:
for column in df.columns:
    print(column)

id
name
alcoholic
category
glassType
instructions
drinkThumbnail
ingredients
ingredientMeasures
text


In [13]:
len(df)

425

In [18]:
df.iloc[0]

id                                                                    0
name                                                                 A1
alcoholic                                                     Alcoholic
category                                                       Cocktail
glassType                                                Cocktail glass
instructions          Pour all ingredients into a cocktail shaker, m...
drinkThumbnail        https://www.thecocktaildb.com/images/media/dri...
ingredients           ['Gin', 'Grand Marnier', 'Lemon Juice', 'Grena...
ingredientMeasures    ['1 3/4 shot ', '1 Shot ', '1/4 Shot', '1/8 Sh...
text                  question Generate a cocktail with Gin Grand Ma...
Name: 0, dtype: object

In [22]:
df.drop(['id'], axis=1, inplace=True)
df.head()

,name,alcoholic,category,glassType,instructions,drinkThumbnail,ingredients,ingredientMeasures,text
0,A1,Alcoholic,Cocktail,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grand Marnier', 'Lemon Juice', 'Grena...","['1 3/4 shot ', '1 Shot ', '1/4 Shot', '1/8 Sh...",question Generate a cocktail with Gin Grand Ma...
1,ABC,Alcoholic,Shot,Shot glass,Layered in a shot glass.,https://www.thecocktaildb.com/images/media/dri...,"['Amaretto', 'Baileys irish cream', 'Cognac']","['1/3 ', '1/3 ', '1/3 ']",question Generate a cocktail with Amaretto Bai...
2,Ace,Alcoholic,Cocktail,Martini Glass,Shake all the ingredients in a cocktail shaker...,https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grenadine', 'Heavy cream', 'Milk', 'E...","['2 shots ', '1/2 shot ', '1/2 shot ', '1/2 sh...",question Generate a cocktail with Gin Grenadin...
3,ACID,Alcoholic,Shot,Shot glass,Poor in the 151 first followed by the 101 serv...,https://www.thecocktaildb.com/images/media/dri...,"['151 proof rum', 'Wild Turkey']","['1 oz Bacardi ', '1 oz ']",question Generate a cocktail with 151 proof ru...
4,Adam,Alcoholic,Ordinary Drink,Cocktail glass,"In a shaker half-filled with ice cubes, combin...",https://www.thecocktaildb.com/images/media/dri...,"['Dark rum', 'Lemon juice', 'Grenadine']","['2 oz ', '1 oz ', '1 tsp ']",question Generate a cocktail with Dark rum Lem...


## Creating a CSV agent 

In [75]:
# Creating an agent with create_csv_agent
agent = create_csv_agent(
    ChatOpenAI(),
    "data/final_cocktails.csv",
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True
)

### Adding memory to the chatbot

In [ ]:
# Creating user input history variable
history = ""

In [80]:
# Checking if user describes his preferences (very basic)
user_input = input()
if "my favorite" in user_input.lower():
    history += user_input + "; "
    
agent.invoke(user_input + " You can also use this info: " + history)

{'input': 'List the ingredients of Clover Club You can also use this info: My favorite ingredients are Gin and Lemon; ',
 'output': 'Gin, Grenadine, Lemon, and Egg white.'}

In [89]:
# creating a FAISS vector database for the history
embeddings = OpenAIEmbeddings()
faiss_index = FAISS.from_texts([""], embeddings)

In [90]:
faiss_index.add_texts(history)

['395fb170-f067-4ba6-b001-c101d9b7b432',
 '31b3429f-20a6-4e19-833c-cb278e9902b0',
 '120ea088-5b88-4adf-8f32-58296b0ce231',
 'c9f70cba-e1da-469c-b062-f868694d0d06',
 'f11d6c49-3c23-4ce7-88e6-e9684e615c0c',
 'fc084ca2-2973-4fd2-9c9b-21e593910099',
 '3240cc86-fe19-4687-aec4-fead0d6c31b1',
 'b11001fe-f997-4ccc-8918-b0ddc3c22420',
 '2d7459c1-0da0-4cea-8c3b-4ff1fbb5a243',
 'fd5890a4-3408-4837-83c6-150e57bf31df',
 '1520384a-31bc-4bdc-9773-6ae7a87d46af',
 '70ec580c-8b6d-449e-b36c-726d74d5f34f',
 'c4acc760-004b-4fff-ba37-5d52e2fd8036',
 '7e34644d-7d04-4ca8-936c-991fa05674b1',
 'cffa74c1-46a2-4f3c-b968-9c3366489273',
 'fa2426e4-c67e-425f-ab9b-0e6e8e5d5b4c',
 '9589f358-a474-474c-b6bf-2f72d5098539',
 '010ccf5b-07db-4042-8157-5306488098ef',
 'b6f33ac5-fab6-4a7b-b5ea-5f0a4e7a042f',
 '5d575f26-cc15-4aff-a146-fad44c04a043',
 'da33ec6a-4590-4345-93b1-b8d5371fa7ee',
 '2763f2ef-8e02-4c82-9be3-f41f9bb95e34',
 'e00ab55b-d341-4260-8616-56f488f4ecf0',
 '867cf1a0-3f40-454d-8dd3-cbd666d14b87',
 '0c910590-ad64-

In [99]:
# Checking if vector database works
stored_texts = [doc.page_content for doc in faiss_index.docstore._dict.values()]
history_loaded = ""
for element in stored_texts:
    history_loaded += element
history_loaded

'My favorite ingredients are Gin and Lemon; '

In [119]:
faiss_index2 = FAISS.load_local("history", embeddings, allow_dangerous_deserialization=True)
stored_texts = [doc.page_content for doc in faiss_index2.docstore._dict.values()]
history_loaded = ""
for element in stored_texts:
    history_loaded += element
history_loaded

''

## Checking memory reliability 

In [114]:
user_input = "My favorite ingredient is lemon."
if "my favorite" in user_input.lower():
    history += user_input + "; "
    
agent.invoke(user_input + " You can also use this info: " + history)["output"]

'There are 104 cocktails in the dataframe that contain lemon in the ingredients.'

In [116]:
user_input = "Recommend me three cocktails which contain ny favorite ingredient."
agent.invoke(user_input + " You can also use this info: " + history)["output"]

'Three cocktails that contain lemon as an ingredient are Dubonnet Cocktail, Lemon Shot, and Shark Attack.'